In [1]:
import sys
sys.path.append('../../practice_demo')

base_lr: 0.000040

In [2]:
from GenerateLabelCPM import *
from modelCPM import *

In [3]:
sym = mxnetModule()

In [4]:
cmodel = intializeModel(sym)

self._data_shapes:  [DataDesc[data,(1, 3, 368, 368),<type 'numpy.float32'>,NCHW]]
self._label_shapes:  [DataDesc[heatmaplabel,(1, 19, 46, 46),<type 'numpy.float32'>,NCHW], DataDesc[partaffinityglabel,(1, 38, 46, 46),<type 'numpy.float32'>,NCHW]]


In [5]:
cocodata = cocoIter('pose_io/data.json',
                    'data', (1,3, 368,368),
                    ['heatmaplabel','partaffinityglabel'],
                    [(1, 19, 46, 46),(1,38,46,46)])

In [6]:
import time
startt = time.time()

cmodel.init_optimizer(optimizer='sgd', 
                      optimizer_params=(('learning_rate', 0.00004), ))

# train one epoch, i.e. going over the data iter one pass
for batch in cocodata:
    cmodel.forward(batch, is_train=True)       # compute predictions
    prediction=cmodel.get_outputs()
    #print prediction
    #print prediction[11].asnumpy().reshape(-1).shape
    #print batch.label[0].asnumpy().reshape(-1).shape
    print '-------------'
    print np.mean((prediction[11].asnumpy().reshape(-1)-batch.label[0].asnumpy().reshape(-1))**2)
    print np.mean((prediction[10].asnumpy().reshape(-1)-batch.label[1].asnumpy().reshape(-1))**2)
    cmodel.backward()                          # compute gradients
    cmodel.update()                            # update parameters using SGD
    
endt = time.time()
print endt-startt

-------------
1.05934e-08
3.5435e-08
-------------
0.000615781
0.00125011
-------------
0.000571712
0.00169049
-------------
0.00196791
0.00841065
-------------
0.00296555
0.0287082
-------------
0.000649921
0.00293906
-------------
0.00079231
0.00291298
-------------
0.000618997
0.00346037
-------------
0.000495606
0.00148083
-------------
2.88742e-06
1.80777e-07
-------------
0.000710441
0.00439732
-------------
0.000111578
0.000323599
-------------
3.63359e-05
2.1786e-06
-------------
0.00113143
0.00535995
-------------
0.000612072
0.00176775
-------------
0.000292848
0.000567834
-------------
0.000487854
0.00164848
-------------
0.000386941
0.00109444
-------------
2.1515e-07
4.21183e-08
-------------
8.94964e-08
4.56508e-08
-------------
0.00032792
0.00330106
-------------
0.00158406
0.00634685
-------------
0.000428516
0.00336787
-------------
0.000347164
0.00344747
-------------
0.000423805
0.00104877
-------------
0.00074387
0.00548392
-------------
0.000325731
0.00109593
-----

In [ ]:
1736/60

### Fake Data

In [ ]:
cocoIterFake = cocoIter('pose_io/data.json',
                    'data', (1,3, 368,368),
                    ['heatmaplabel','partaffinityglabel'],
                    [(1, 19, 46, 46),(1,38,46,46)])

In [ ]:
import time
startt = time.time()

cmodel.init_optimizer(optimizer='sgd', 
                      optimizer_params=(('learning_rate', 0.01), ))

# train one epoch, i.e. going over the data iter one pass
for batch in cocoIterFake:
    cmodel.forward(batch, is_train=True)       # compute predictions
    prediction=cmodel.get_outputs()
    #print prediction
    #print prediction[11].asnumpy().reshape(-1).shape
    #print batch.label[0].asnumpy().reshape(-1).shape
    print '-------------'
    print np.mean((prediction[11].asnumpy().reshape(-1)-batch.label[0].asnumpy().reshape(-1))**2)
    print np.mean((prediction[10].asnumpy().reshape(-1)-batch.label[1].asnumpy().reshape(-1))**2)
    cmodel.backward()                          # compute gradients
    cmodel.update()                            # update parameters using SGD
    
endt = time.time()
print endt-startt

### Comparsion

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline

cocodata = cocoIter('pose_io/data.json',
                    'data', (1,3, 368,368),
                    ['heatmaplabel','partaffinityglabel'],
                    [(1, 19, 46, 46),(1,38,46,46)])

dataitera = cocodata.next()

newsimage = dataitera.data[0].asnumpy()[0]

heatmap1 = dataitera.label[0].asnumpy()[0]

pagmap1 = dataitera.label[1].asnumpy()[0]

#oriImg[:,:,[2,1,0]]
f = plt.imshow(newsimage[0,:,:]) # reorder it before displaying

plt.figure();  # plt.axis('off')

In [ ]:
heatmap1.shape

In [ ]:
pagmap1.shape

In [ ]:
import matplotlib

In [ ]:
for i in range(1,2):
    plt.imshow(cv.resize(heatmap1[i,:,:], (368, 368)))

plt.imshow(newsimage[0,:,:], alpha=0.5)

fig = matplotlib.pyplot.gcf()
fig.set_size_inches(5, 5)

plt.show()

In [ ]:
cmodel.forward(dataitera)       # compute predictions
prediction=cmodel.get_outputs()

heatmap2 = prediction[11].asnumpy()[0]

plt.figure();  # plt.axis('off')

for i in range(18,19):
    plt.imshow(cv.resize(heatmap2[i,:,:], (368, 368)), alpha=(1 - 1.0 * i / 10))

plt.imshow(newsimage[0,:,:], alpha=0.1)

plt.show()

In [ ]:
differentmap = heatmap2[7,:,:]-heatmap1[7,:,:]

plt.figure();  # plt.axis('off')

plt.imshow(cv.resize(differentmap, (368, 368)))

plt.imshow(newsimage[0,:,:], alpha=0.1)

plt.show()

In [ ]:
differentmap[0,1]

In [ ]:
differentmap = heatmap2[10,:,:]-heatmap1[10,:,:]

plt.figure();  # plt.axis('off')

plt.imshow(cv.resize(differentmap, (368, 368)))

plt.imshow(newsimage[0,:,:], alpha=0.1)

plt.show()

In [ ]:
differentmap = heatmap2[18,:,:]-heatmap1[18,:,:]

plt.figure();  # plt.axis('off')

plt.imshow(cv.resize(differentmap, (368, 368)))

plt.imshow(newsimage[0,:,:], alpha=0.1)

plt.show()